<a href="https://colab.research.google.com/github/shengy90/MSc-Project/blob/master/notebooks/26th_July_Agglo_Cluster_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1️⃣ Setup Notebook 💻**


### **Authenticate with BigQuery ☁️**

In [ ]:
!pip install --upgrade google-cloud-bigquery[bqstorage,pandas]
!pip install --upgrade pandas-gbq

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
%%bigquery --project machine-learning-msc df --use_bqstorage_api
SELECT 
  COUNT(*) as total_rows
FROM `machine-learning-msc.low_carbon_london.household_consumption_daily_agg` 

In [3]:
df.head()

,total_rows
0,14841792


### **Importing Libraries⏬**

##### Standard Libraries

In [4]:
!pip install fbprophet

In [5]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import random
import datetime as dt

from tqdm import tqdm
from datetime import date
from matplotlib.gridspec import GridSpec
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
 
sns.set()
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


##### Import Github Repository

In [6]:
%cd /content
!ls

/content
adc.json  drive  mscproj  sample_data


In [7]:
!rm -rf mscproj
!git clone https://github.com/shengy90/MSc-Project mscproj
!git pull
%cd /content/mscproj/
!ls

Cloning into 'mscproj'...
remote: Enumerating objects: 306, done.
remote: Counting objects: 100% (306/306), done.
remote: Compressing objects: 100% (215/215), done.
remote: Total 306 (delta 164), reused 204 (delta 83), pack-reused 0
Receiving objects: 100% (306/306), 10.79 MiB | 12.30 MiB/s, done.
Resolving deltas: 100% (164/164), done.
fatal: not a git repository (or any of the parent directories): .git
/content/mscproj
bin	     __init__.py  notebooks  requirements.txt  sql
definitions  Makefile	  README.md  run.py	       src


In [8]:
%reload_ext autoreload 
%autoreload 2 
from src.train_prophet import TrainProphet

# **2️⃣ Download Datasets from BigQuery**

### **Downloading data for Baseline Model from BQ**

##### Baseline model train dataset

In [9]:
%%bigquery --project machine-learning-msc df_baseline_train --use_bqstorage_api
SELECT 
train.ts AS ds,
ROUND(avg(train.kwhh),3) AS y,
COUNT(DISTINCT train.lcl_id) AS households_num,
MAX(weather.air_temperature) AS air_temperature

FROM `machine-learning-msc.forecasting_20200719.train_set` train
LEFT JOIN `machine-learning-msc.london_heathrow_hourly_weather_data.london_heathrow_hourly_weather` weather 
ON TIMESTAMP_TRUNC(weather.ts, HOUR) = TIMESTAMP_TRUNC(train.ts, hour)

-- Only include data from November -> February
-- We'll use November -> January data to forecast February (28 days X 48 data points a day)
WHERE CAST(train.ts AS DATE) >= '2012-11-01' AND  CAST(train.ts AS DATE) < '2013-03-01'

GROUP BY 1 
ORDER BY 1 ASC

##### Baseline model test dataset

In [10]:
%%bigquery --project machine-learning-msc df_baseline_test --use_bqstorage_api
SELECT 
train.ts AS ds,
ROUND(avg(train.kwhh),3) AS y,
COUNT(DISTINCT train.lcl_id) AS households_num,
MAX(weather.air_temperature) AS air_temperature

FROM `machine-learning-msc.forecasting_20200719.test_set` train
LEFT JOIN `machine-learning-msc.london_heathrow_hourly_weather_data.london_heathrow_hourly_weather` weather 
ON TIMESTAMP_TRUNC(weather.ts, HOUR) = TIMESTAMP_TRUNC(train.ts, hour)

-- Only include data from November -> February
-- We'll use November -> January data to forecast February (28 days X 48 data points a day)
WHERE CAST(train.ts AS DATE) >= '2012-11-01' AND  CAST(train.ts AS DATE) < '2013-03-01'

GROUP BY 1 
ORDER BY 1 ASC

##### Format data

In [11]:
df_baseline_train['ds'] = df_baseline_train['ds'].dt.tz_localize(None) # remove timezones 
df_baseline_test['ds'] = df_baseline_test['ds'].dt.tz_localize(None) # remove timezones 

df_baseline_train.head(), df_baseline_test.head()

(                   ds      y  households_num  air_temperature
 0 2012-11-01 00:00:00  0.185            2680             11.8
 1 2012-11-01 00:30:00  0.195            2680             11.8
 2 2012-11-01 01:00:00  0.170            2680              8.8
 3 2012-11-01 01:30:00  0.154            2680              8.8
 4 2012-11-01 02:00:00  0.138            2676              8.5,
                    ds      y  households_num  air_temperature
 0 2012-11-01 00:00:00  0.189             999             11.8
 1 2012-11-01 00:30:00  0.208            1000             11.8
 2 2012-11-01 01:00:00  0.185            1000              8.8
 3 2012-11-01 01:30:00  0.165            1000              8.8
 4 2012-11-01 02:00:00  0.149            1000              8.5)

### **Download data for Agglo Clusters from BQ**

##### agglo training dataset

In [12]:
%%bigquery --project machine-learning-msc df_agglo_train --use_bqstorage_api

WITH pca_clusters AS (
  SELECT 
  DISTINCT 
  lcl_id,
  cluster,
  cluster_x,
  cluster_y
  FROM `machine-learning-msc.low_carbon_london.pca_clusters_20200712`
  ),
  
cluster_data AS (
  SELECT 
  pca.cluster,
  pca.cluster_x,
  pca.cluster_y,
  train.*,
  weather.air_temperature
  FROM `machine-learning-msc.forecasting_20200719.train_set` train
  LEFT JOIN pca_clusters pca ON pca.lcl_id = train.lcl_id
  LEFT JOIN `machine-learning-msc.london_heathrow_hourly_weather_data.london_heathrow_hourly_weather` weather 
  ON TIMESTAMP_TRUNC(weather.ts, HOUR) = TIMESTAMP_TRUNC(train.ts, hour)
  
  WHERE CAST(train.ts AS DATE) >= '2012-11-01' AND  CAST(train.ts AS DATE) < '2013-03-01'
  )

SELECT 
cluster,
ts AS ds,
ROUND(AVG(kwhh),4) AS y,
MAX(air_temperature) AS air_temperature,
COUNT(DISTINCT lcl_id) AS households_num
FROM cluster_data

GROUP BY 1,2
ORDER BY 1,2

##### agglo test set

In [13]:
%%bigquery --project machine-learning-msc df_agglo_test --use_bqstorage_api

WITH pca_clusters AS (
  SELECT 
  DISTINCT 
  lcl_id,
  cluster,
  cluster_x,
  cluster_y
  FROM `machine-learning-msc.low_carbon_london.pca_clusters_20200712`
  ),
  
cluster_data AS (
  SELECT 
  pca.cluster,
  pca.cluster_x,
  pca.cluster_y,
  train.*,
  weather.air_temperature
  FROM `machine-learning-msc.forecasting_20200719.test_set` train
  LEFT JOIN pca_clusters pca ON pca.lcl_id = train.lcl_id
  LEFT JOIN `machine-learning-msc.london_heathrow_hourly_weather_data.london_heathrow_hourly_weather` weather 
  ON TIMESTAMP_TRUNC(weather.ts, HOUR) = TIMESTAMP_TRUNC(train.ts, hour)
  
  WHERE CAST(train.ts AS DATE) >= '2012-11-01' AND  CAST(train.ts AS DATE) < '2013-03-01'
  )

SELECT 
cluster,
ts AS ds,
ROUND(AVG(kwhh),4) AS y,
MAX(air_temperature) AS air_temperature,
COUNT(DISTINCT lcl_id) AS households_num
FROM cluster_data

GROUP BY 1,2
ORDER BY 1,2

##### format data

In [14]:
df_agglo_train['ds'] = df_agglo_train['ds'].dt.tz_localize(None) # remove timezones 
df_agglo_test['ds'] = df_agglo_test['ds'].dt.tz_localize(None) # remove timezones 

df_agglo_train.groupby('ds').mean().head(10), df_agglo_test.groupby('ds').mean().head(10)

(                            y  air_temperature  households_num
 ds                                                            
 2012-11-01 00:00:00  0.610500             11.8      297.777778
 2012-11-01 00:30:00  1.130056             11.8      297.777778
 2012-11-01 01:00:00  1.022156              8.8      297.777778
 2012-11-01 01:30:00  0.934922              8.8      297.777778
 2012-11-01 02:00:00  0.780467              8.5      297.333333
 2012-11-01 02:30:00  0.681878              8.5      297.666667
 2012-11-01 03:00:00  0.636444              7.8      297.777778
 2012-11-01 03:30:00  0.617911              7.8      297.777778
 2012-11-01 04:00:00  0.583856              7.1      297.777778
 2012-11-01 04:30:00  0.549433              7.1      297.777778,
                             y  air_temperature  households_num
 ds                                                            
 2012-11-01 00:00:00  0.472189             11.8      111.000000
 2012-11-01 00:30:00  0.988578         

### **Download data for SOM clusters from BQ**

##### som training dataset

In [15]:
%%bigquery --project machine-learning-msc df_som_train --use_bqstorage_api

WITH 

som_cluster_num AS (
  SELECT cluster_x, cluster_y,
  CAST(ROW_NUMBER() OVER (ORDER BY cluster_x, cluster_y)-1 AS STRING) AS cluster
  FROM `machine-learning-msc.low_carbon_london.som_clusters_20200712`
  WHERE train_test_split = 'train'
  group by 1,2
  order by 1,2 asc
  ),
  
som_clusters AS (
  SELECT 
  DISTINCT 
  a.lcl_id,
  b.cluster,
  a.cluster_x,
  a.cluster_y
  FROM `machine-learning-msc.low_carbon_london.som_clusters_20200712` a 
  LEFT JOIN som_cluster_num b ON a.cluster_x = b.cluster_x AND a.cluster_y = b.cluster_y
  ),

cluster_data AS (
  SELECT 
  som.cluster,
  som.cluster_x,
  som.cluster_y,
  train.*,
  weather.air_temperature
  FROM `machine-learning-msc.forecasting_20200719.train_set` train
  LEFT JOIN som_clusters som ON som.lcl_id = train.lcl_id
  LEFT JOIN `machine-learning-msc.london_heathrow_hourly_weather_data.london_heathrow_hourly_weather` weather 
  ON TIMESTAMP_TRUNC(weather.ts, HOUR) = TIMESTAMP_TRUNC(train.ts, hour)
  
  WHERE CAST(train.ts AS DATE) >= '2012-11-01' AND  CAST(train.ts AS DATE) < '2013-03-01'
  )

SELECT 
cluster,
ts AS ds,
ROUND(AVG(kwhh),4) AS y,
ROUND(SUM(kwhh), 4) AS total_kwhh,
MAX(air_temperature) AS air_temperature,
COUNT(DISTINCT lcl_id) AS households_num
FROM cluster_data

GROUP BY 1,2
ORDER BY 1,2

##### som test set

In [16]:
%%bigquery --project machine-learning-msc df_som_test --use_bqstorage_api

WITH 

som_cluster_num AS (
  SELECT cluster_x, cluster_y,
  CAST(ROW_NUMBER() OVER (ORDER BY cluster_x, cluster_y)-1 AS STRING) AS cluster
  FROM `machine-learning-msc.low_carbon_london.som_clusters_20200712`
  WHERE train_test_split = 'test'
  group by 1,2
  order by 1,2 asc
  ),
  
som_clusters AS (
  SELECT 
  DISTINCT 
  a.lcl_id,
  b.cluster,
  a.cluster_x,
  a.cluster_y
  FROM `machine-learning-msc.low_carbon_london.som_clusters_20200712` a 
  LEFT JOIN som_cluster_num b ON a.cluster_x = b.cluster_x AND a.cluster_y = b.cluster_y
  ),

cluster_data AS (
  SELECT 
  som.cluster,
  som.cluster_x,
  som.cluster_y,
  train.*,
  weather.air_temperature
  FROM `machine-learning-msc.forecasting_20200719.test_set` train
  LEFT JOIN som_clusters som ON som.lcl_id = train.lcl_id
  LEFT JOIN `machine-learning-msc.london_heathrow_hourly_weather_data.london_heathrow_hourly_weather` weather 
  ON TIMESTAMP_TRUNC(weather.ts, HOUR) = TIMESTAMP_TRUNC(train.ts, hour)
  
  WHERE CAST(train.ts AS DATE) >= '2012-11-01' AND  CAST(train.ts AS DATE) < '2013-03-01'
  )

SELECT 
cluster,
ts AS ds,
ROUND(AVG(kwhh),4) AS y,
ROUND(SUM(kwhh), 4) AS total_kwhh,
MAX(air_temperature) AS air_temperature,
COUNT(DISTINCT lcl_id) AS households_num
FROM cluster_data

GROUP BY 1,2
ORDER BY 1,2

##### format data

In [17]:
df_som_train['ds'] = df_som_train['ds'].dt.tz_localize(None) # remove timezones 
df_som_test['ds'] = df_som_test['ds'].dt.tz_localize(None) # remove timezones 

df_som_train.groupby('ds').mean().head(10), df_som_test.groupby('ds').mean().head(10)

(                            y  total_kwhh  air_temperature  households_num
 ds                                                                        
 2012-11-01 00:00:00  0.214467   55.002556             11.8      297.777778
 2012-11-01 00:30:00  0.229322   58.101111             11.8      297.777778
 2012-11-01 01:00:00  0.199556   50.510444              8.8      297.777778
 2012-11-01 01:30:00  0.179867   45.733556              8.8      297.777778
 2012-11-01 02:00:00  0.160822   41.094000              8.5      297.333333
 2012-11-01 02:30:00  0.144544   37.313667              8.5      297.666667
 2012-11-01 03:00:00  0.134667   34.895222              7.8      297.777778
 2012-11-01 03:30:00  0.129956   33.504222              7.8      297.777778
 2012-11-01 04:00:00  0.127622   33.137667              7.1      297.777778
 2012-11-01 04:30:00  0.129944   33.433333              7.1      297.777778,
                             y  total_kwhh  air_temperature  households_num
 ds        

# **3️⃣ Training Models**

### **Training baseline forecast**

In [18]:
baseline_model = TrainProphet("2013-02-01")
baseline_model.fit(df_baseline_train)
baseline_model.evaluate_test_global_mape(df_baseline_test)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


Training Mean Absolute Percentage Error: 7.454017857142844
Test Mean Absolute Percentage Error: 8.88


### **Training Agglo Clusters**

In [50]:
def train_clusters(df_train, df_test, test_period="2013-02-01"):
    forecast_dict = {}
    test_global_fc = pd.DataFrame()
    train_global_fc = pd.DataFrame()
    clusters = df_train.groupby('cluster').count().index.to_list()
    for cluster in clusters:
        cluster_dict = {} 
        print(f"\nTraining cluster: {cluster}") 
        print("---------------------------")
        df_train_cluster = df_train.query(f"cluster=='{cluster}'").copy()
        df_test_cluster = df_test.query(f"cluster=='{cluster}'").copy()
        model = TrainProphet(test_period)
        model.fit(df_train_cluster)
        model.evaluate_test_global_mape(df_test_cluster)
        cluster_dict['model'] = model 
        forecast_dict[f'cluster_{cluster}']=cluster_dict
        test_global_fc = pd.concat([test_global_fc, model.test_forecast])

        train_forecast = df_train[['cluster','ds','y']].copy()
        train_forecast['max_households'] = df_train['households_num'].max()
        train_forecast = train_forecast.merge(model.forecast[['ds', 'yhat']], left_on='ds', right_on='ds')
        train_forecast['y_global'] = train_forecast['y'] * train_forecast['max_households']
        train_forecast['yhat_global'] = train_forecast['yhat'] * train_forecast['max_households']
        train_global_fc = pd.concat([train_global_fc, train_forecast])

    return forecast_dict, test_global_fc, train_global_fc

In [51]:
agglo_dict, agglo_global_test, agglo_global_train = train_clusters(df_agglo_train, df_agglo_test)


Training cluster: 0
---------------------------
Training Mean Absolute Percentage Error: 6.713616071428577
Test Mean Absolute Percentage Error: 72.09

Training cluster: 1
---------------------------
Training Mean Absolute Percentage Error: 10.567767857142881
Test Mean Absolute Percentage Error: 68.99

Training cluster: 2
---------------------------
Training Mean Absolute Percentage Error: 93.82300595238095
Test Mean Absolute Percentage Error: 74.17

Training cluster: 3
---------------------------
Training Mean Absolute Percentage Error: 9.677522321428553
Test Mean Absolute Percentage Error: 376.27000000000004

Training cluster: 4
---------------------------
Training Mean Absolute Percentage Error: 9.18360863095236
Test Mean Absolute Percentage Error: 11.4

Training cluster: 5
---------------------------
Training Mean Absolute Percentage Error: 1733264479330.5093
Test Mean Absolute Percentage Error: 108.16

Training cluster: 6
---------------------------
Training Mean Absolute Percenta

##### Evaluate Test/ Train MAPE

In [52]:
agglo_test = agglo_global_test.groupby('ds')[['households_num', 'y_global','yhat_global']].sum()
agglo_test_global_mape = np.round(np.mean(np.abs(agglo_test['yhat_global']/agglo_test['y_global']-1)),4)*100
print(f"agglo_test_global_mape: {agglo_test_global_mape}")

agglo_test_global_mape: 298.53000000000003


In [53]:
agglo_train = agglo_global_train.groupby('ds')[['y_global','yhat_global']].sum()
agglo_train_global_mape = np.round(np.mean(np.abs(agglo_train['yhat_global']/agglo_train['y_global']-1)),4)*100
print(f"agglo_train_global_mape: {agglo_train_global_mape}")

agglo_train_global_mape: 10.489999999999998


### **Training SOM Clusters**

##### Training Forecast

In [55]:
som_dict, som_global_test, som_global_train = train_clusters(df_som_train, df_som_test)


Training cluster: 0
---------------------------
Training Mean Absolute Percentage Error: 6.837886904761923
Test Mean Absolute Percentage Error: 52.28

Training cluster: 1
---------------------------
Training Mean Absolute Percentage Error: 8.251034226190475
Test Mean Absolute Percentage Error: 17.36

Training cluster: 2
---------------------------
Training Mean Absolute Percentage Error: 9.76328125000001
Test Mean Absolute Percentage Error: 18.990000000000002

Training cluster: 3
---------------------------
Training Mean Absolute Percentage Error: 12.772157738095249
Test Mean Absolute Percentage Error: 25.28

Training cluster: 4
---------------------------
Training Mean Absolute Percentage Error: 17.782291666666655
Test Mean Absolute Percentage Error: 34.64

Training cluster: 5
---------------------------
Training Mean Absolute Percentage Error: 12.562418154761891
Test Mean Absolute Percentage Error: 35.86

Training cluster: 6
---------------------------
Training Mean Absolute Percent

##### Evaluate Test Train Mape

In [56]:
som_test = som_global_test.groupby('ds')[['y_global','yhat_global']].sum()
som_test_global_mape = np.round(np.mean(np.abs(som_test['yhat_global']/som_test['y_global']-1)),4)*100
print(f"som_test_global_mape: {som_test_global_mape}")

som_test_global_mape: 8.41


In [57]:
som_train = som_global_train.groupby('ds')[['y_global','yhat_global']].sum()
som_train_global_mape = np.round(np.mean(np.abs(som_train['yhat_global']/som_train['y_global']-1)),4)*100
print(f"som_train_global_mape: {som_train_global_mape}")

som_train_global_mape: 7.9


# **4️⃣ Save Predictions to BQ**

In [44]:
import pandas_gbq
def output_to_bq(forecast, table_id, project_id='machine-learning-msc'):
    pandas_gbq.to_gbq(forecast, table_id, project_id=project_id)

In [43]:
output_to_bq(baseline_model.forecast[['ds','yhat']], table_id='forecasting_20200719.baseline_train_prediction')
output_to_bq(baseline_model.test_forecast[['ds','yhat']], table_id='forecasting_20200719.baseline_test_prediction')

5760 out of 5760 rows loaded.
1it [00:03,  3.64s/it]
1344 out of 1344 rows loaded.
1it [00:04,  4.79s/it]


In [58]:
output_to_bq(agglo_global_train[['ds','yhat','cluster']], table_id='forecasting_20200719.agglo_train_prediction')
output_to_bq(agglo_global_test[['ds','yhat','cluster']], table_id='forecasting_20200719.agglo_test_prediction')

466560 out of 466560 rows loaded.
1it [00:44, 44.47s/it]
12096 out of 12096 rows loaded.
1it [00:06,  6.92s/it]


In [59]:
output_to_bq(som_global_train[['ds','yhat','cluster']], table_id='forecasting_20200719.som_train_prediction')
output_to_bq(som_global_test[['ds','yhat','cluster']], table_id='forecasting_20200719.som_test_prediction')

466560 out of 466560 rows loaded.
1it [00:26, 26.11s/it]
12096 out of 12096 rows loaded.
1it [00:03,  3.34s/it]
